In [49]:
import json
import pandas as pd
import numpy as np
from utils import *
import datasets
from scipy.stats import pearsonr, spearmanr

SYNTHETIC_LABELS = '/home/abdelrahman.sadallah/mbzuai/review_rewrite/inference/evalute_outputs/adapters/SciLitLLM/score_only/instruction/step-843/review_evaluation_human_annotation'
gold_data  = pd.read_excel(f'chatgpt/outputs/main_data_batch_gold_results.xlsx', sheet_name=None)
aspects = [ 'actionability', 'grounding_specificity','verifiability', 'helpfulness']

data = {}

for aspect in aspects:
    data[aspect] = []
    fine_tuning_outputs = []
    predictions_path = SYNTHETIC_LABELS + f'/predictions_{aspect}_gold.jsonl'
    with open(predictions_path, 'r') as f:
        for line in f:
            l = json.loads(line)
            fine_tuning_outputs.append(l[f'{aspect}_label'])

    for i in range(len(fine_tuning_outputs)):
        sample = {}
        sample['fine_tuning_output'] = fine_tuning_outputs[i]

        if fine_tuning_outputs[i] == 'None':
            continue

        # if fine_tuning_outputs[i] == 'X':
        #     fine_tuning_outputs[i] = 6

        sample['chatgpt_output'] = gold_data[aspect].iloc[i][f'chatgpt_{aspect}_score']
        sample['gold'] = gold_data[aspect].iloc[i][f'{aspect}_label']
        ## get the number of charachters in the review point
        length = len(gold_data[aspect].iloc[i]['review_point'].split())
        sample['length'] = length

        # if length < 100:
        #     continue


        data[aspect].append(sample)


sheets = {}
with open(f'length_labels_correlation_goldset.txt', 'w') as f:
    for aspect in aspects:
        df = pd.DataFrame(data[aspect])
        df = df.dropna()

        #3 remove rows that has X for any column
        # df = df[df['fine_tuning_output'] != 'X']
        # df = df[df['chatgpt_output'] != 'X']
        # df = df[df['gold'] != 'X']

        ## convert the value of X to 0 in any column
        df = df.replace({'X': 0})

        sheets[aspect] = df

        
        f.write(f'Aspect: {aspect}\n')
        for col in df.columns:
            if col == 'length':
                continue

            ### convert the columns to int
            df[col] = df[col].astype(int)
            df['length'] = df['length'].astype(int)
            f.write(f'Correlation between length and {col}: {df["length"].corr(df[col])}\n')
            pearson_corr, _ = pearsonr(df['length'].to_list(), df[col].to_list())
            spearman_corr, _ = spearmanr(df['length'], df[col])
            f.write(f'Pearson correlation between length and {col}: {pearson_corr}\n')
            f.write(f'Spearman correlation between length and {col}: {spearman_corr}\n')
            f.write('\n')
        f.write('\n\n')
        


### write to excel file
with pd.ExcelWriter('length_labels_correlation_goldset.xlsx') as writer:
    for aspect in aspects:
        sheets[aspect].to_excel(writer, sheet_name=aspect)


In [50]:
import json
import pandas as pd
import numpy as np
from utils import *
import datasets

SYNTHETIC_LABELS = '/home/abdelrahman.sadallah/mbzuai/review_rewrite/inference/evalute_outputs/adapters/SciLitLLM/score_only/instruction/step-843/review_evaluation_automatic_labels'
aspects = [ 'actionability', 'grounding_specificity','verifiability', 'helpfulness']

data = {}

for aspect in aspects:
    raw_data = datasets.load_dataset('boda/review_evaluation_automatic_labels', aspect, split='test').to_pandas()

    data[aspect] = []
    fine_tuning_outputs = []
    predictions_path = SYNTHETIC_LABELS + f'/predictions_all_test.jsonl'
    with open(predictions_path, 'r') as f:
        for line in f:
            l = json.loads(line)
            fine_tuning_outputs.append(l[f'{aspect}_label'])

    for i in range(len(fine_tuning_outputs)):
        sample = {}
        sample['fine_tuning_output'] = fine_tuning_outputs[i]

        if fine_tuning_outputs[i] == 'None':
            continue

        # if fine_tuning_outputs[i] == 'X':
        #     fine_tuning_outputs[i] = 6

        
        sample['chatgpt'] = raw_data.iloc[i][f'chatgpt_{aspect}_score']
        ## get the number of charachters in the review point
        length = len(raw_data.iloc[i]['review_point'].split())
        sample['length'] = length
        data[aspect].append(sample)


sheets = {}
with open(f'length_labels_correlation_testset.txt', 'w') as f:
    for aspect in aspects:
        df = pd.DataFrame(data[aspect])

        print(len(df))
        # #3 remove rows that has X for any column
        # df = df[df['fine_tuning_output'] != 'X']
        # df = df[df['chatgpt'] != 'X']   
        # print(len(df))

        ## convert the value of X to 0 in any column
        df = df.replace({'X': 0})

        sheets[aspect] = df

        
        f.write(f'Aspect: {aspect}\n')
        for col in df.columns:
            if col == 'length':
                continue
            f.write(f'Correlation between length and {col}: {df["length"].corr(df[col])}\n')


# ### write to excel file
# with pd.ExcelWriter('length_labels_correlation_goldset.xlsx') as writer:
#     for aspect in aspects:
#         sheets[aspect].to_excel(writer, sheet_name=aspect)


1000
1000
1000
1000
